# Run Inference on Llama-3-8B-Instruct

## References
1. [Model Card on HuggingFace](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct)

## Install and Load Dependencies

In [1]:
!pip install accelerate

In [2]:
import torch
from datetime import datetime
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from huggingface_hub import notebook_login as hfl

2024-06-11 17:04:12.624209: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-11 17:04:12.624270: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-11 17:04:12.625814: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
hfl()

## Import Model and Tokenizer

In [ ]:
checkpoint = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)

model = AutoModelForCausalLM.from_pretrained(
    checkpoint,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
print(f"Memory footprint: {model.get_memory_footprint() / 1e6:.2f} MB")

## Create Test Prompt

In [ ]:
prompt = """
Write a script in Python that implements a circular queue and creates one with user input. Give an example of how the script works and document the code with appropriate comments and docstrings wherever necessary. 
Generate a response only for the prompt that is given and nothing else. 
"""

In [ ]:
messages = [
    {"role": "system", "content": "You are a helpful and masterful programming assistant that generates markdown documentation for any input program."},
    {"role": "user", "content": prompt},
]

## Run Inference with `model.generate()`

In [ ]:
input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

In [ ]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [ ]:
%%time
outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]

In [ ]:
print(tokenizer.decode(response, skip_special_tokens=True))

## Test Generate with Custom Input

In [ ]:
language = "Python"
code = """
def test_euler_method(self):
    def dydx(x, y):
        return x + y

    x0, y0 = 0, 1
    x_end = 1
    h = 0.01
    x, y = differential_equations.euler_method(dydx, x0, y0, x_end, h)
    expected = np.exp(x_end) - x_end - 1  # Analytical solution for the differential equation dy/dx = x + y
    self.assertAlmostEqual(y[-1], expected, places=2)
"""

In [ ]:
template = f"""
Given a script file in {language}, generate its documentation for each function. For each function in the script, document its name, arguments, return values, and a brief explanation of its logic. Ensure that code within comments is not parsed and documented. Generate nothing else than what is asked.

Strictly use the following format for each function:

## Function Name: `function_name`

### Arguments
* `arg1` (type): Description of argument 1.
* `arg2` (type): Description of argument 2.
* ...

### Return Values
* `return_value1` (type): Description of return value 1.
* `return_value2` (type): Description of return value 2.
* ...

### Explanation of Function Logic:
1. Brief explanation of the function logic step by step.
2. ...
3. ...

-----------------------------------------------------------------------------

Here is the function I want you to generate documentation of:

{code}
"""

In [ ]:
template

In [ ]:
output = generateOutputWithModelPipeline(model, tokenizer, template)
generatedText = output[0]['generated_text']
response = generatedText[len(template):].strip()

In [ ]:
print(generatedText)

In [ ]:
print(response)

## Remarks

I'm sure StarCoder2 is great in it's own regard as a code completion agent, however, for my use case, it simply won't cut it.